In [ ]:
from typing import ClassVar, Optional, Annotated
from datetime import datetime, UTC

from pydantic import BaseModel, computed_field, ConfigDict, Field
from pydantic_firestore import FirestoreModel, FirestoreDict
from pydantic_firestore.transforms import *


class Nested(BaseModel):
    first_field: str
    second_field: str

    class Update(BaseModel):
        first_field: Optional[str] = None
        second_field: Optional[str] = None
        
        @computed_field
        def update_time(self) -> FirestoreTimestamp:
            return FIRESTORE_TIMESTAMP


class Doc(FirestoreModel):
    simple_nested: Nested
    integer_value: int
    float_value: float
    string_array: list[str]
    create_time: datetime
    update_time: datetime | None = None
    custom_time: datetime | None = None

    firestore_config: ClassVar = FirestoreDict(location="pydantic-firestore")

    class Create(FirestoreModel):
        simple_nested: Nested = Field(
            default_factory=lambda: Nested(first_field="first", second_field="second")
        )
        integer_value: int = 0
        float_value: float = 0.0
        string_array: list[str] = []
        custom_time: datetime | None = None

        model_config = ConfigDict(arbitrary_types_allowed=True)

        @computed_field
        @property
        def create_time(self) -> FirestoreTimestamp:
            return FIRESTORE_TIMESTAMP

        @computed_field
        @property
        def update_time(self) -> None:
            return None

    class Update(FirestoreModel):
        simple_nested: Optional[Nested.Update] = None
        integer_value: Optional[FirestoreSentinel[int]] = None
        float_value: Optional[FirestoreSentinel[float]] = None
        string_array: Optional[FirestoreSentinel[list[str]]] = None

        custom_time: Optional[datetime | FirestoreTimestamp] = None

        model_config = ConfigDict(arbitrary_types_allowed=True)

        @computed_field
        def update_time(self) -> FirestoreTimestamp:
            return FIRESTORE_TIMESTAMP

In [ ]:
from google.cloud.firestore import Client

db = Client()

In [ ]:
Doc.Create(
    simple_nested=Nested(first_field="Foo", second_field="Bar"),
    integer_value=42,
    float_value=3.14,
    string_array=["foo"],
).create_to_firestore(db, "example")

In [ ]:
snapshot = Doc.firestore_read(db, "example")
snapshot

In [ ]:
Doc.Update(simple_nested=Nested.Update(first_field="first")).update_to_firestore(db,  "example")

In [ ]:
Doc.Update(string_array=(FIRESTORE_UNION, ["kek", "foo", "bar"])).update_to_firestore(db,  "example")

In [ ]:
Doc.Update(integer_value=(FIRESTORE_INCREMENT, +100)).update_to_firestore(db, "example")

In [ ]:
Doc.Update(custom_time=datetime.now(UTC)).update_to_firestore(db, "example")

In [ ]:
Doc.Update(custom_time=FIRESTORE_TIMESTAMP).update_to_firestore(db, "example")

In [ ]:
Doc.firestore_delete(db, "example")